# Automated Factor Process System 

need to define:

In [3]:
# define symbol to collect data
symbol, interval, delay_minutes, start_year, start_month, start_day, end_year, end_month, end_day = 'BTCUSDT', '1h', 1, 2021, 1, 1, 2024, 6, 1
### (end not included) #such as: 'BTCUSDT', '1h', 1, 2021, 1, 1, 2024, 6, 1

# calculate return

# feature processing method
method = 'standardize' 
### method = normalize/standardize

# train_test_split
train_start, train_end, test_start, test_end = '2021-01-01', '2024-01-01', '2024-01-01', '2024-06-01' # can change

# simply strategy threshold and cost
threshold = 0.002
cost = 0.002

## 1. Data Collection

In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import a_binance_data_collection as binance_data_collection
import b1_calculate_return as cr
import b2_class_calculate_return as ccr
import c1_calculate_specific_factor as csf # we will change feature engineering(factor)
import d_feature_preprocssing as feapre
import e_train_test_split as tts 
import model_construction_1_linear as model_construction_1_linear # we will change model
import f1_basic_strategy_backtest as bsb # we will change stategy
import f2_basic_strategy_backtesting_class as bsbc

In [5]:
# binance_data = binance_data_collection.binance_data_collection(symbol, interval, delay_minutes, start_year, start_month, start_day, end_year, end_month, end_day) # symbol, interval, delay_minutes, start_year, start_month, start_day, end_year, end_month, end_day(end not included)

In [6]:
data  = pd.read_csv('BTCUSDT_2021_1_1_to_2024_6_1_with_1h.csv')
data['date_time'] = pd.to_datetime(data['date_time'])
data['real_time'] = pd.to_datetime(data['real_time'])
data['vwap'] = (data['close'] * data['volume']).cumsum() / data['volume'].cumsum()
print(data)

# simply process
for column in data.columns:
    median_value = data[column].median()
    data[column] = data[column].fillna(median_value)
print(data.isna().sum())

                date_time           real_time      timestamp      open  \
0     2021-01-01 00:01:00 2021-01-01 00:00:00  1609459200000  28948.19   
1     2021-01-01 01:01:00 2021-01-01 01:00:00  1609462800000  29015.00   
2     2021-01-01 02:01:00 2021-01-01 02:00:00  1609466400000  29443.65   
3     2021-01-01 03:01:00 2021-01-01 03:00:00  1609470000000  29236.58   
4     2021-01-01 04:01:00 2021-01-01 04:00:00  1609473600000  29302.11   
...                   ...                 ...            ...       ...   
29923 2024-05-31 19:01:00 2024-05-31 19:00:00  1717182000000  67456.20   
29924 2024-05-31 20:01:00 2024-05-31 20:00:00  1717185600000  67720.00   
29925 2024-05-31 21:01:00 2024-05-31 21:00:00  1717189200000  67755.00   
29926 2024-05-31 22:01:00 2024-05-31 22:00:00  1717192800000  67602.50   
29927 2024-05-31 23:01:00 2024-05-31 23:00:00  1717196400000  67537.20   

           high       low     close     volume  quote_asset_volume  \
0      29055.00  28706.00  29015.00   803

In [7]:
# # plot data
# def plot_data(data, x_column, y_column, title):
#     fig = go.Figure()
#     fig.add_trace(go.Scatter(x=data[x_column], y=data[y_column], mode='lines', name=y_column))
#     fig.update_layout(title=title, xaxis_title=x_column, yaxis_title=y_column)
#     fig.show()

# for column in data.columns[3:14]:
#     plot_data(data, 'date_time', column, f'{column}')

## 2. Factor calcualtion 

In [8]:

# calcualte return
period_labels = [interval]
data = cr.compute_returns(data, period_labels=period_labels)

# period_labels = [interval]
# data = ccr.compute_returns(data, period_labels=period_labels, cost = cost)
# print(data)

In [9]:
# caculate 101 worldquant factors
import c2_101Alpha as c2_101Alpha
df = data.copy()
df_fac = c2_101Alpha.get_alpha(df)
print(df_fac)

d:\app\anaconda\envs\gputorch\lib\site-packages\pandas\core\arraylike.py:396: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
d:\app\anaconda\envs\gputorch\lib\site-packages\pandas\core\arraylike.py:396: RuntimeWarning: invalid value encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
d:\OneDrive\algohero\Automated Factor Process System_0722_0727\c2_101Alpha.py:386: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df['alpha094']=stock.alpha094()
d:\OneDrive\algohero\Automated Factor Process System_0722_0727\c2_101Alpha.py:387: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joinin

                              real_time      timestamp      open      high  \
date_time                                                                    
2021-01-01 00:01:00 2021-01-01 00:00:00  1609459200000  28948.19  29055.00   
2021-01-01 01:01:00 2021-01-01 01:00:00  1609462800000  29015.00  29546.42   
2021-01-01 02:01:00 2021-01-01 02:00:00  1609466400000  29443.65  29500.00   
2021-01-01 03:01:00 2021-01-01 03:00:00  1609470000000  29236.58  29379.41   
2021-01-01 04:01:00 2021-01-01 04:00:00  1609473600000  29302.11  29422.32   
...                                 ...            ...       ...       ...   
2024-05-31 19:01:00 2024-05-31 19:00:00  1717182000000  67456.20  68059.40   
2024-05-31 20:01:00 2024-05-31 20:00:00  1717185600000  67720.00  67834.40   
2024-05-31 21:01:00 2024-05-31 21:00:00  1717189200000  67755.00  67763.00   
2024-05-31 22:01:00 2024-05-31 22:00:00  1717192800000  67602.50  67708.80   
2024-05-31 23:01:00 2024-05-31 23:00:00  1717196400000  67537.20

d:\OneDrive\algohero\Automated Factor Process System_0722_0727\c2_101Alpha.py:392: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df['alpha100']=stock.alpha100()
d:\OneDrive\algohero\Automated Factor Process System_0722_0727\c2_101Alpha.py:393: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df['alpha101']=stock.alpha101()


In [10]:

# # calculate specific factor
# df = data.copy()
# period_labels = period_labels

# start_idx = df.columns.get_loc('volume') + 1
# return_col = df.filter(like='return').columns.tolist()
# end_idx = min(df.columns.get_loc(col) for col in return_col)
# specific_factor_col = df.columns[start_idx:end_idx]

# df_fac = csf.calculate_specific_factor(df, specific_factor_col, period_labels = period_labels)
# # print(df_fac)


In [11]:
# from tsfresh import extract_features
# from tsfresh.utilities.dataframe_functions import impute
# from tsfresh.utilities.distribution import MultiprocessingDistributor
# from tsfresh import extract_features
# from tsfresh.utilities.dataframe_functions import roll_time_series
# train_start, train_end, test_start, test_end = train_start, train_end, test_start, test_end
# train_df, test_df, Y, X, Y_train, X_train, Y_test, X_test = tts.train_test_split(df_fac, train_start, train_end, test_start, test_end)
# timeseries = train_df[['close', 'volume', 'quote_asset_volume', 'number_of_trades', 'taker_buy_base_asset_volume', 'taker_buy_quote_asset_volume']].copy()
# timeseries['code'] = 1
# timeseries.reset_index(inplace=True)

# data_roll = roll_time_series(timeseries, column_id='code', column_sort='date_time', max_timeshift=3, min_timeshift=1).drop(columns=['code'])


# distributor = MultiprocessingDistributor(n_workers=1, disable_progressbar=False, progressbar_title="Feature Extraction")
    
# extracted_features = extract_features(data_roll, column_id="id", column_sort="date_time", chunksize=60,distributor=distributor)
# impute(extracted_features)

# print(extracted_features)
# extracted_features.to_csv('extracted_features.csv')

In [12]:

# calculate commom factor
## use tsfresh

## use featuretools
### soon....

## 3.Feature preprocessing 

In [13]:
feature_df = df_fac.copy()

# return_data & feature_data processing
return_col = feature_df.filter(like='return').columns.tolist()
feature_df = feapre.handle_return(feature_df, return_col)
max_return_index = max(feature_df.columns.get_loc(col) for col in return_col)
feature_col = feature_df.columns[max_return_index + 1:].tolist()
feature_df = feapre.feature_preprocessing(feature_df, feature_col, method = method) # method = normalize/standardize

# print(feature_df)

Are there any missing values in the DataFrame?  False


## 4.Model construction 

In [14]:
# # model_df = feature_df.copy()
# # model_df = model_df.to_csv('a.csv')
# model_df = pd.read_csv('a.csv')
# model_df['date_time'] = pd.to_datetime(model_df['date_time'])
# model_df['real_time'] = pd.to_datetime(model_df['real_time'])
# model_df = model_df.set_index('date_time')
# model_df

In [15]:
model_df = feature_df.copy()
train_start, train_end, test_start, test_end = train_start, train_end, test_start, test_end
train_df, test_df, Y, X, Y_train, X_train, Y_test, X_test = tts.train_test_split(model_df, train_start, train_end, test_start, test_end)


In [16]:
import model_construction_1_linear as model_construction_1_linear
result_df, test_df = model_construction_1_linear.model_construction_linear(train_df, test_df, Y, X, Y_train, X_train, Y_test, X_test)

d:\OneDrive\algohero\Automated Factor Process System_0722_0727\model_construction_1_linear.py:52: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  test_df[f'{x_col}___{y_col}_pred_rtn'] = Y_test_pred
d:\OneDrive\algohero\Automated Factor Process System_0722_0727\model_construction_1_linear.py:52: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  test_df[f'{x_col}___{y_col}_pred_rtn'] = Y_test_pred
d:\OneDrive\algohero\Automated Factor Process System_0722_0727\model_construction_1_linear.py:52: PerformanceWarning: DataFrame is highly fr

In [17]:
# import model_construction_2_lasso as model_construction_2_lasso
# result_df, test_df = model_construction_2_lasso.model_construction_lasso(train_df, test_df, Y, X, Y_train, X_train, Y_test, X_test)

In [18]:
# import model_construction_3_ridge as model_construction_3_ridge
# result_df, test_df = model_construction_3_ridge.model_construction_ridge(train_df, test_df, Y, X, Y_train, X_train, Y_test, X_test)

In [19]:
# import model_construction_4_logistic as model_construction_4_logistic
# result_df, test_df = model_construction_4_logistic.model_construction_logistic(train_df, test_df, Y, X, Y_train, X_train, Y_test, X_test)

In [20]:
# import model_construction_5_lda as model_construction_5_lda
# result_df, test_df = model_construction_5_lda.model_construction_lda(train_df, test_df, Y, X, Y_train, X_train, Y_test, X_test)

In [21]:
# import model_construction_6_qda as model_construction_6_qda
# result_df, test_df = model_construction_6_qda.model_construction_qda(train_df, test_df, Y, X, Y_train, X_train, Y_test, X_test)

In [22]:
# import model_construction_7_decision_trees_reg as model_construction_7_decision_trees_reg
# result_df, test_df = model_construction_decision_trees(train_df, test_df, Y, X, Y_train, X_train, Y_test, X_test)

In [23]:
# import model_construction_8_decision_trees_class as model_construction_8_decision_trees_class
# result_df, test_df = model_construction_decision_trees(train_df, test_df, Y, X, Y_train, X_train, Y_test, X_test)

In [24]:
# import model_construction_9_random_forest_reg as model_construction_9_random_forest_reg
# result_df, test_df = model_construction_9_random_forest_reg.model_construction_random_forest(train_df, test_df, Y, X, Y_train, X_train, Y_test, X_test)

In [25]:
# import model_construction_10_random_forest_class as model_construction_10_random_forest_class
# result_df, test_df = model_construction_10_random_forest_class.model_construction_random_forest(train_df, test_df, Y, X, Y_train, X_train, Y_test, X_test)

In [26]:
# import model_construction_11_xgboost_reg as model_construction_11_xgboost_reg
# result_df, test_df = model_construction_11_xgboost_reg.model_construction_xgboost(train_df, test_df, Y, X, Y_train, X_train, Y_test, X_test)

In [27]:
# import model_construction_12_xgboost_class as model_construction_12_xgboost_class
# result_df, test_df = model_construction_12_xgboost_class.model_construction_xgboost(train_df, test_df, Y, X, Y_train, X_train, Y_test, X_test)

In [28]:
# import model_construction_13_adaboost_reg as model_construction_13_adaboost_reg
# result_df, test_df = model_construction_13_adaboost_reg.model_construction_adaboost(train_df, test_df, Y, X, Y_train, X_train, Y_test, X_test)

In [29]:
# import model_construction_14_adaboost_class as model_construction_14_adaboost_class
# result_df, test_df = model_construction_14_adaboost_class.model_construction_adaboost(train_df, test_df, Y, X, Y_train, X_train, Y_test, X_test)

In [30]:
# import model_construction_15_gbm_reg as model_construction_15_gbm_reg
# result_df, test_df = model_construction_15_gbm_reg.model_construction_gbm(train_df, test_df, Y, X, Y_train, X_train, Y_test, X_test)

In [31]:
# import model_construction_16_gbm_class as model_construction_16_gbm_class
# result_df, test_df = model_construction_16_gbm_class.model_construction_gbm(train_df, test_df, Y, X, Y_train, X_train, Y_test, X_test)

In [32]:
result_df

,Y,X,Lowest_Loss,R2_Scores,Coef_params
102,return_1h,alpha100,0.000034,[0.0008055866495360053],[[0.00010618118910934547]]
64,return_1h,alpha060,0.000034,[0.0006618859461586801],[[9.177375823889586e-05]]
55,return_1h,alpha052,0.000034,[0.0005971785574017119],[[-8.05527524807928e-05]]
29,return_1h,alpha028_2,0.000034,[0.0005731079278209483],[[6.842969661029797e-05]]
57,return_1h,alpha054_1,0.000034,[0.00041200603192192986],[[0.00010934826244075748]]


In [33]:
columns_to_keep = ['close']
for y_unique in result_df['Y'].unique():
    columns_to_keep.append(y_unique)
for x in result_df['X'].unique():
    for y in result_df[result_df['X'] == x]['Y'].unique():
        pred_col_name = f"{x}___{y}_pred_rtn"
        if pred_col_name not in columns_to_keep:
            columns_to_keep.append(pred_col_name)
        
model_df = test_df[columns_to_keep]
model_df

,close,return_1h,alpha100___return_1h_pred_rtn,alpha060___return_1h_pred_rtn,alpha052___return_1h_pred_rtn,alpha028_2___return_1h_pred_rtn,alpha054_1___return_1h_pred_rtn
date_time,,,,,,,
2024-01-01 00:01:00,42503.5,0.004478,0.000065,0.000014,3.024337e-05,0.000006,-0.000028
2024-01-01 01:01:00,42647.9,0.003397,0.000083,0.000034,3.919318e-05,0.000041,0.000043
2024-01-01 02:01:00,42620.4,-0.000645,0.000082,0.000020,4.366998e-05,0.000031,-0.000008
2024-01-01 03:01:00,42369.8,-0.005880,0.000097,0.000055,4.641100e-05,0.000091,0.000129
2024-01-01 04:01:00,42436.6,0.001577,0.000061,-0.000037,4.369474e-05,-0.000019,-0.000156
...,...,...,...,...,...,...,...
2024-05-31 18:01:00,67456.1,0.001119,0.000062,0.000010,5.007781e-05,-0.000022,-0.000106
2024-05-31 19:01:00,67719.9,0.003911,0.000041,0.000036,5.603091e-07,0.000043,0.000047
2024-05-31 20:01:00,67755.0,0.000518,0.000072,0.000022,3.537899e-05,0.000028,-0.000002


## 5.Backtesting

In [34]:
back_df = model_df.copy()
back_df = bsb.basic_strategy_backtesting(back_df, threshold, cost)
# back_df = bsbc.basic_strategy_backtesting(back_df, cost)
back_df

Skipping alpha100___return_1h_pred_rtn_cum_rtn as it only contains zero values.
Skipping alpha060___return_1h_pred_rtn_cum_rtn as it only contains zero values.
Skipping alpha052___return_1h_pred_rtn_cum_rtn as it only contains zero values.
Skipping alpha028_2___return_1h_pred_rtn_cum_rtn as it only contains zero values.
Skipping alpha054_1___return_1h_pred_rtn_cum_rtn as it only contains zero values.
Skipping.


,close,return_1h,alpha100___return_1h_pred_rtn,alpha060___return_1h_pred_rtn,alpha052___return_1h_pred_rtn,alpha028_2___return_1h_pred_rtn,alpha054_1___return_1h_pred_rtn,alpha100___return_1h_pred_rtn_sta_rtn,alpha100___return_1h_pred_rtn_cum_rtn,alpha060___return_1h_pred_rtn_sta_rtn,alpha060___return_1h_pred_rtn_cum_rtn,alpha052___return_1h_pred_rtn_sta_rtn,alpha052___return_1h_pred_rtn_cum_rtn,alpha028_2___return_1h_pred_rtn_sta_rtn,alpha028_2___return_1h_pred_rtn_cum_rtn,alpha054_1___return_1h_pred_rtn_sta_rtn,alpha054_1___return_1h_pred_rtn_cum_rtn,bas_cum_rtn
date_time,,,,,,,,,,,,,,,,,,
2024-01-01 00:01:00,42503.5,0.004478,0.000065,0.000014,3.024337e-05,0.000006,-0.000028,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.004478
2024-01-01 01:01:00,42647.9,0.003397,0.000083,0.000034,3.919318e-05,0.000041,0.000043,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.007876
2024-01-01 02:01:00,42620.4,-0.000645,0.000082,0.000020,4.366998e-05,0.000031,-0.000008,-0.0,0.0,-0.0,0.0,-0.0,0.0,-0.0,0.0,-0.0,0.0,0.007231
2024-01-01 03:01:00,42369.8,-0.005880,0.000097,0.000055,4.641100e-05,0.000091,0.000129,-0.0,0.0,-0.0,0.0,-0.0,0.0,-0.0,0.0,-0.0,0.0,0.001351
2024-01-01 04:01:00,42436.6,0.001577,0.000061,-0.000037,4.369474e-05,-0.000019,-0.000156,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.002928
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-05-31 18:01:00,67456.1,0.001119,0.000062,0.000010,5.007781e-05,-0.000022,-0.000106,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.528960
2024-05-31 19:01:00,67719.9,0.003911,0.000041,0.000036,5.603091e-07,0.000043,0.000047,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.532871
2024-05-31 20:01:00,67755.0,0.000518,0.000072,0.000022,3.537899e-05,0.000028,-0.000002,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.533389
